In [1]:
def int_bin(integer):
    return '{0:04b}'.format(integer)


def str_bin(string):
    return " ".join(f"{ord(i):08b}" for i in string)


def hex_bin(string):
    return " ".join(
        reversed([i + j for i, j in zip(*[["{0:04b}".format(int(c, 16)) for c in reversed("0" + string)][n::2] for n
                                          in [1, 0]])]))


def bin_hex(string):
    return hex(int(string, 2))


def bin_str(string):
    string_list = split_in(string)
    return "".join([chr(int(binary, 2)) for binary in string_list])


def split_in(block, char=8):
    return [block[i:i + char] for i in range(0, len(block), char)]


def get_first_and_last_bit(bits6):
    two_bits = bits6[0] + bits6[-1]
    return int(two_bits, 2)


def get_middle_four_bit(bits6):
    four_bits = bits6[1:5]
    return int(four_bits, 2)


def split_block(block):
    k1 = ""
    k2 = ""
    for i in range(len(block)):
        if i < (len(block) / 2):
            k1 += block[i]
        else:
            k2 += block[i]
    return k1, k2

In [2]:
def permutation(string, arr_perm):
    per = ""
    for i in arr_perm:
        per += string[i - 1]
    return per


def shift_left(string, shifts):
    s = ""
    for i in range(shifts):
        for j in range(1, 28):
            s += string[j]
        s += string[0]
        string = s
        s = ""
    return string


def xor(text1, text2):
    ans = ""
    for i in range(len(text1)):
        if text1[i] == text2[i]:
            ans += "0"
        else:
            ans += "1"
    return ans


def rounds(gi, di, ki):
    return di, xor(util_func(di, ki), gi)


def sub_keys(key):
    key_bin1 = hex_bin(key)
    key_bin1 = key_bin1.replace(" ", "")
    key_bin_perm = permutation(key_bin1, key_perm1)
    key1, key2 = split_block(key_bin_perm)
    subkeys = [shift_left(key1, i) + shift_left(key2, i) for i in shift_list]
    subkeys = [permutation(i, key_perm2) for i in subkeys]
    return subkeys


def substitution(block):
    block_chain_6bits = split_in(block, char=6)
    block_chain_4bits = []
    block_chain_4bits += [
        int_bin(SBOX_DES[i][get_first_and_last_bit(block_chain_6bits[i])][get_middle_four_bit(block_chain_6bits[i])])
        for i in range(len(block_chain_6bits))]
    return "".join(block_chain_4bits)


def util_func(di, ki):
    return permutation(substitution(xor(permutation(di, expansion), ki)), perm_32bit)

In [3]:
def cipher(key64bit, message, form_key, form_msg):
    subkey = sub_keys(key=key64bit, form="str_str")
    message = hex_bin(message.replace(" ", ""))
    mess_block = split_in(message.replace(" ", ""), char=64)
    ciphered = ""
    for mes in mess_block:
        g0, d0 = split_block(permutation(mes.replace(" ", ""), initial_perm))
        gn, dn = g0, d0
        for x in range(16):
            gn, dn = rounds(gn, dn, subkey[x])
        final_msg = dn + "" + gn
        ciphered += bin_str(permutation(final_msg, final_perm)) if form_msg == "str" else bin_hex(
            permutation(final_msg, final_perm)) + " "
    return ciphered[2:]

In [4]:
print(cipher("133457799BBCDFF1", "0123456789ABCDEF", "hex", "hex"))

85e813540f0ab405 


In [5]:
def decipher(key64bit, message, form_key, form_msg):
    initial_perm = [58, 50, 42, 34, 26, 18, 10, 2,
                    60, 52, 44, 36, 28, 20, 12, 4,
                    62, 54, 46, 38, 30, 22, 14, 6,
                    64, 56, 48, 40, 32, 24, 16, 8,
                    57, 49, 41, 33, 25, 17, 9, 1,
                    59, 51, 43, 35, 27, 19, 11, 3,
                    61, 53, 45, 37, 29, 21, 13, 5,
                    63, 55, 47, 39, 31, 23, 15, 7]
    final_perm = [40, 8, 48, 16, 56, 24, 64, 32, 39, 7, 47, 15,
                  55, 23, 63, 31, 38, 6, 46, 14, 54, 22, 62, 30,
                  37, 5, 45, 13, 53, 21, 61, 29, 36, 4, 44, 12,
                  52, 20, 60, 28, 35, 3, 43, 11, 51, 19, 59, 27,
                  34, 2, 42, 10, 50, 18, 58, 26, 33, 1, 41, 9,
                  49, 17, 57, 25]
    subkey = sub_keys(key)
    message = hex_bin(message.replace(" ", ""))
    subkey.reverse()
    mess_block = split_in(message.replace(" ", ""), char=64)
    ciphered = ""
    for mes in mess_block:
        g0, d0 = split_block(permutation(mes.replace(" ", ""), initial_perm))
        gn, dn = g0, d0
        for x in range(16):
            gn, dn = rounds(gn, dn, subkey[x])
        final_msg = dn + "" + gn
        ciphered += bin_str(permutation(final_msg, final_perm)) if form_msg == "str" else bin_hex(
            permutation(final_msg, final_perm)) + " "
    return ciphered[2:]

In [6]:
print(decipher("133457799BBCDFF1", "85E813540F0AB405", "hex", "hex"))

123456789abcdef 
